# Merger Rate of Primordial Black Holes in Dwarf Galaxy
## Using monocrhomatic masses  $$m_{PBH} = 1 M_{\odot}$$


#### Code Writed by Tadeo D.

**References:** [arXiv:2105.14523](https://arxiv.org/pdf/2105.14523.pdf). [arXiv:1603.05234](https://arxiv.org/pdf/1603.05234.pdf)

In [1]:
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl
import math
import seaborn as sns
from scipy import stats
from astropy import constants as const
import csv
from scipy import constants
from scipy.integrate import odeint
from tqdm import tqdm
from scipy import integrate
from scipy.integrate import quad, dblquad
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u
from itertools import combinations, permutations
from itertools import combinations_with_replacement
from astropy.cosmology import Planck13, z_at_value

In [2]:
#Edad del universo en el redshift z = 20
z_20 = cosmo.age(20)
#La función cosmo.age(z) regresa el valor de la edad del universo dado un valor de redshift
#Tiempo disponible para realizar las multiples coalescencias
time_enable = cosmo.age(0)-z_20 

In [3]:
#Tiempos asignados para cada etapa de coalescencia
t_1_gn = 3.3e+09
t_2_gn = 2.3e+09
t_3_gn = 3.3e+09
t_4_gn = 4.7e+09

In [4]:
#Declaración de constantes
H0 = cosmo.H(0) #Parametro de Hubble
G = const.G #Constante Gravitacional
c = const.c #velocidad de la luz
pi = math.pi #numero pi

In [5]:
m_i = 1.*u.M_sun
m_j = 1.*u.M_sun

In [6]:
#Declaración de valores del Cluster
M = 10**5 * u.M_sun
R = 0.9*u.parsec
v_rel = (G*M/R)**(1./2.)
interval = .1
r = np.arange(.1, R.value+interval, interval)

In [7]:
if m_i==m_j:
    N_i = M/(2*m_i)
    N_j = M/(2*m_j)

if m_j>m_i or m_i>m_j:
    porcent_1 = input("Add the mass percentage existent in the form of m_i BH:\t")#percent
    porcent_2 = 100.0 - float(porcent_1)
    N_i = (M * (float(porcent_1)/100.0))/m_i
    N_j = (M * (float(porcent_2)/100.0))/m_j

M_i = m_i*N_i
M_j = m_j*N_j

etiqueta_mass = ["m_i", "m_j"]
mass_catalog = [m_i.value, m_j.value]
M_tot_catalog = [M_i, M_j]
N_tot_catalog = [N_i, N_j]
etiqueta_M_tot = ["M_i","M_j"]
array_0gnd = []
array_0gnd.append(N_i + N_j)
array_0gnd.append(0)

array_M_0 = []
array_M_0.append(M_i + M_j)
array_M_0.append(0)

array_M_1 = []
array_M_2 = []
array_M_3 = []
array_M_4 = []

array_1st =[]
array_2nd =[]
array_3rd =[]
array_4th =[]

In [8]:
def sigma(mass1,mass2):
    sigma_result = 2*math.pi*(85*math.pi/(6*math.sqrt(2)))**(2./7.) * (G**(2.) * (mass1+mass2)**(10./7.) * mass1**(2./7.) * mass2**(2./7.))/(c**(10./7.))
    return sigma_result.to(u.parsec**(32./7)/u.yr**(18./7.))

def R_s(M):
    return(2*G*M)/(c**2)

def Total_Mass(m_1, m_2):
    a_0 = R_s(m_1).to(u.AU) + R_s(m_2).to(u.AU)
    a_1=0.1*u.AU
    E_ij = (G*m_1*m_2/2)*(1/a_1 - 1/a_0)
    return (m_1+m_2) + E_ij/(c**2.)

def GAMMA(r1,r0):
    a=(r1**(2.)/((R.value)**(2.)))
    b=(r0**(2.)/(R**(2.)))
    return (r1**(2.))*((1.+(a))**(-9./4.))*(1./((1. + a)**(1./4.)) + 1./((1. + b)**(1./4.)))**(3./7.) - abs((1./((1. + a)**(1./4.)) - 1./((1. + b)**(1./4.))))**(3./7.)

def phi_3(r, M_cluster):
    return (G*M_cluster/R)*((1 + r**(2.)/R**(2.))**(-1./2.))

def n_3(r, m_sp, M_sp):#m_sp mass specie
    return (rho_3(r, M_sp)/m_sp)

def rho_3(r, M_sp): #M_sp Total Mass de especie
    return ((3*M_sp)/(4*(math.pi)*(R**(3.))))*((1 + r**(2.)/R**(2.))**(-5./2.))

def vel_3(r, N_sp, m_sp, M_sp, M_cluster): #N_sp numero de especie
    return (alfa**(1./2.))*(N_sp**(1./2.))*((phi_3(r, M_cluster)**(3))/(n_3(r,m_sp,M_sp)**(1./2.)))

def integrand_n_i(r1):
    a=(r1**(2.)/((R.value)**(2.)))
    return (r1**(2.))*((1 + a)**(-5./2.))

final_mass = []
M_t_final = []
etiqueta_final = []
N_t_final = []



t_f_m = []
generation=1
index = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
#print(list(combinations_with_replacement(etiqueta_mass,2)),"\n")
while generation<2:
    M_cluster = 0
    for i in M_tot_catalog:
        M_cluster += i
    print("GENERATION NUMBER", generation)
    print("M_cluster", M_cluster)
    print(list(combinations(etiqueta_mass,2)),"\n")
    cont = 0
    sum_N = 0
    for i,j,k in zip(range(len(list(combinations(mass_catalog,2)))),range(len(list(combinations(etiqueta_mass,2)))), range(len(list(combinations(N_tot_catalog,2))))):
        mass_input = list(combinations(mass_catalog,2))[i]
        name_mass = list(combinations(etiqueta_mass,2))[j]
        N_tot_input = list(combinations(N_tot_catalog,2))[k]
        name_m_1 = name_mass[0]
        m_1=mass_input[0]*u.M_sun
        N_1 = N_tot_input[0]
        name_m_2 = name_mass[1]
        m_2=mass_input[1]*u.M_sun
        N_2 = N_tot_input[1]
        m_final = Total_Mass(m_1, m_2)
        final_mass.append(m_final)
        cont_pbh = 0
        new_label_mass = str("m_")+str(generation)+str(index[cont])
        #print(name_m_1, "\t ", name_m_2,"\t ", "sigma(",name_m_1,",",name_m_2,")")
        #print("{:.2f}".format(m_1.value),"\t","{:.2f}".format(m_2.value),"\t","{:.2e}".format(sigma(m_1,m_2)),"\n")
        #print("N(",name_m_1,")","\t\t N(",name_m_2,")")
        N_sp_1 = N_1
        M_sp_1 = N_sp_1 * m_1
        N_sp_2 = N_2
        M_sp_2 = N_sp_2 * m_2
        #print("{:.1f}".format(N_sp_1),"\t\t\t {:.1f}".format(N_sp_2),"\n\n")
        #print("Radius\t| Merger_Rate \t\t| Time to the first Merger | Probability \t| Collision Number")
        #print("--------------------------------------------------------------------------------------------")
        alfa = ((4*24*(2**(1/2))*(R**(2.)))/(math.pi*(G**(5.))*(M_cluster**(5.))*(2**(17./2.))))
        betha = (3./(4*math.pi*(R**(3.))))
        etha = (G*M_cluster/R)
        for h in r :
            r0 = h*u.parsec
            I, err = quad(GAMMA, h+interval, 0.,args=(r0,))
            ctes = (14*math.pi/3.)*sigma(m_1,m_2)*(alfa**(-2./7.))*(betha**(9./7.))*(etha**(-12./7.))*(N_sp_1)*(n_3(r0, m_2, M_sp_2)/vel_3(r0, N_sp_2, m_2, M_sp_2, M_cluster))
            merger = (I*ctes*(u.parsec**(3.))).to(u.yr**(-1))
            GAMMA_value = merger
            Prob = GAMMA_value*(t_1_gn*u.yr)#integration dt
            Int_res, err = quad(integrand_n_i, h , h+interval)
            N_sp_r = 3.*(2.**(3./2.))*N_sp_1*(R**(-3.))*Int_res*u.parsec**3.
            Num_Col = N_sp_r*Prob #numero de choques especie i-j
            cont_pbh = cont_pbh + round(float(Num_Col))
            #print("{0:.2f}".format(h),"\t|","{:e}".format(GAMMA_value), "|\t" , "{0:.2f}".format((merger**(-1.)).to(u.Gyr)),"\t\t|\t","{:.4f}".format(Prob), " \t|\t" , "{:.4f}".format(Num_Col), " \t | ")
        #print("\n")
        #print(round(float(cont_pbh)), "PBHs generates of", "{0:.1f}".format(m_final), "denoted by", new_label_mass)
        #print("\n")
        cont+=1
        N_t_final.append(cont_pbh)
        M_t_final.append(cont_pbh*m_final)   
        etiqueta_final.append(new_label_mass)    
    for i, j, k, l in zip(final_mass, M_t_final, etiqueta_final, N_t_final):
        mass_catalog.append(i.value)
        M_tot_catalog.append(j)
        etiqueta_mass.append(k)
        N_tot_catalog.append(l)
    generation +=1
    #print(etiqueta_mass)
    #print(mass_catalog)
    #print(N_tot_catalog)
    final_mass.clear()
    M_t_final.clear()
    N_t_final.clear()
    etiqueta_final.clear()
    #print("\n")

GENERATION NUMBER 1
M_cluster 100000.0 solMass
[('m_i', 'm_j')] 



In [9]:
N_tot_catalog[0] = N_tot_catalog[0] - (N_tot_catalog[2])
N_tot_catalog[1] = N_tot_catalog[1] - (N_tot_catalog[2])
#print(N_tot_catalog)

In [10]:
N_tot_catalog[0] = 2*N_tot_catalog[1]
N_tot_catalog.pop(1)
M_tot_catalog.pop(1)
mass_catalog.pop(1)
etiqueta_mass.pop(1)
M_tot_catalog[0] = N_tot_catalog[0]*(mass_catalog[0]*u.M_sun)
M_tot_catalog[1] = N_tot_catalog[1]*(mass_catalog[1]*u.M_sun)
#print(N_tot_catalog)
#print(M_tot_catalog)
for i in M_tot_catalog:
    array_M_1.append(i)
array_M_0.append(0)
array_M_1.append(0)
for i in N_tot_catalog:
    array_1st.append(i)
array_1st.append(0)
array_0gnd.append(0)

In [11]:
while generation<3:
    M_cluster = 0
    for i in M_tot_catalog:
        M_cluster += i
    print("GENERATION NUMBER", generation)
    print("M_cluster", M_cluster)
    print(list(combinations_with_replacement(etiqueta_mass,2)),"\n")
    cont = 0
    sum_N = 0
    for i,j,k in zip(range(len(list(combinations_with_replacement(mass_catalog,2)))),range(len(list(combinations_with_replacement(etiqueta_mass,2)))), range(len(list(combinations_with_replacement(N_tot_catalog,2))))):
        mass_input = list(combinations_with_replacement(mass_catalog,2))[i]
        name_mass = list(combinations_with_replacement(etiqueta_mass,2))[j]
        N_tot_input = list(combinations_with_replacement(N_tot_catalog,2))[k]
        name_m_1 = name_mass[0]
        m_1=mass_input[0]*u.M_sun
        N_1 = N_tot_input[0]
        name_m_2 = name_mass[1]
        m_2=mass_input[1]*u.M_sun
        N_2 = N_tot_input[1]
        m_final = Total_Mass(m_1, m_2)
        final_mass.append(m_final)
        cont_pbh = 0
        new_label_mass = str("m_")+str(generation)+str(index[cont])
        #print(name_m_1, "\t ", name_m_2,"\t ", "sigma(",name_m_1,",",name_m_2,")")
        #print("{:.2f}".format(m_1.value),"\t","{:.2f}".format(m_2.value),"\t","{:.2e}".format(sigma(m_1,m_2)),"\n")
        #print("N(",name_m_1,")","\t\t N(",name_m_2,")")
        if m_1==m_2:
            N_sp_1 = N_1/2.
            M_sp_1 = N_sp_1 * m_1
            N_sp_2 = N_2/2.
            M_sp_2 = N_sp_2 * m_2
        else:
            N_sp_1 = N_1
            M_sp_1 = N_sp_1 * m_1
            N_sp_2 = N_2
            M_sp_2 = N_sp_2 * m_2
        #print("{:.1f}".format(N_sp_1),"\t\t\t {:.1f}".format(N_sp_2),"\n\n")
        #print("Radius\t| Merger_Rate \t\t| Time to the first Merger | Probability \t| Collision Number")
        #print("--------------------------------------------------------------------------------------------")
        alfa = ((4*24*(2**(1/2))*(R**(2.)))/(math.pi*(G**(5.))*(M_cluster**(5.))*(2**(17./2.))))
        betha = (3./(4*math.pi*(R**(3.))))
        etha = (G*M_cluster/R)
        for h in r :
            r0 = h*u.parsec
            I, err = quad(GAMMA, h+interval, 0., args=(r0,))
            ctes = (14*math.pi/3.)*sigma(m_1,m_2)*(alfa**(-2./7.))*(betha**(9./7.))*(etha**(-12./7.))*(N_sp_1)*(n_3(r0, m_2, M_sp_2)/vel_3(r0, N_sp_2, m_2, M_sp_2, M_cluster))
            merger = (I*ctes*(u.parsec**(3.))).to(u.yr**(-1))
            GAMMA_value = merger
            Prob = GAMMA_value*(t_2_gn*u.yr)#integration dt
            Int_res, err = quad(integrand_n_i, h , h+interval)
            N_sp_r = 3.*(2.**(3./2.))*N_sp_1*(R**(-3.))*Int_res*u.parsec**3.
            Num_Col = N_sp_r*Prob #numero de choques especie i-j
            cont_pbh = cont_pbh + round(float(Num_Col))
            #print("{0:.2f}".format(h),"\t|","{:e}".format(GAMMA_value), "|\t" , "{0:.2f}".format((merger**(-1.)).to(u.Gyr)),"\t\t|\t","{:.4f}".format(Prob), " \t|\t" , "{:.4f}".format(Num_Col), " \t | ")
        #print("\n")
        #print(round(float(cont_pbh)), "PBHs generates of", "{0:.1f}".format(m_final), "denoted by", new_label_mass)
        #print("\n")
        cont+=1
        N_t_final.append(cont_pbh)
        M_t_final.append(cont_pbh*m_final)   
        etiqueta_final.append(new_label_mass)    
    for i, j, k, l in zip(final_mass, M_t_final, etiqueta_final, N_t_final):
        mass_catalog.append(i.value)
        M_tot_catalog.append(j)
        etiqueta_mass.append(k)
        N_tot_catalog.append(l)
    generation +=1
    #print(etiqueta_mass)
    #print(mass_catalog)
    #print(N_tot_catalog)
    final_mass.clear()
    M_t_final.clear()
    N_t_final.clear()
    etiqueta_final.clear()
    #print("\n")

GENERATION NUMBER 2
M_cluster 99352.87525550122 solMass
[('m_i', 'm_i'), ('m_i', 'm_1a'), ('m_1a', 'm_1a')] 



In [12]:
N_tot_catalog[0] = N_tot_catalog[0] - (2*N_tot_catalog[2] + N_tot_catalog[3])
N_tot_catalog[1] = N_tot_catalog[1] - (2*N_tot_catalog[4] + N_tot_catalog[3])
N_tot_catalog[1] = N_tot_catalog[1] + N_tot_catalog[2]
N_tot_catalog.pop(2)
mass_catalog.pop(2)
etiqueta_mass.pop(2)
M_tot_catalog.pop(2)

<Quantity 4899.37512896 solMass>

In [13]:
zeros = []
for i in range(len(N_tot_catalog)):
    if N_tot_catalog[i] == 0:
        zeros.append(i)
        
for i in range(len(zeros)):
    N_tot_catalog.pop(zeros[0])
    M_tot_catalog.pop(zeros[0])
    mass_catalog.pop(zeros[0])
    etiqueta_mass.pop(zeros[0])

for i in range(len(M_tot_catalog)):
    M_tot_catalog[i] = N_tot_catalog[i]*(mass_catalog[i]*u.M_sun)
    
for i in M_tot_catalog:
    array_M_2.append(i)
array_M_0.append(0)
array_M_1.append(0)
array_M_2.append(0)

In [14]:
for i in N_tot_catalog:
    array_2nd.append(i)
array_2nd.append(0)
array_0gnd.append(0)
array_1st.append(0)

In [15]:
while generation<4:
    M_cluster = 0
    for i in M_tot_catalog:
        M_cluster += i
    print("GENERATION NUMBER", generation)
    print("M_cluster", M_cluster)
    print(list(combinations_with_replacement(etiqueta_mass,2)),"\n")
    cont = 0
    sum_N = 0
    for i,j,k in zip(range(len(list(combinations_with_replacement(mass_catalog,2)))),range(len(list(combinations_with_replacement(etiqueta_mass,2)))), range(len(list(combinations_with_replacement(N_tot_catalog,2))))):
        mass_input = list(combinations_with_replacement(mass_catalog,2))[i]
        name_mass = list(combinations_with_replacement(etiqueta_mass,2))[j]
        N_tot_input = list(combinations_with_replacement(N_tot_catalog,2))[k]
        name_m_1 = name_mass[0]
        m_1=mass_input[0]*u.M_sun
        N_1 = N_tot_input[0]
        name_m_2 = name_mass[1]
        m_2=mass_input[1]*u.M_sun
        N_2 = N_tot_input[1]
        m_final = Total_Mass(m_1, m_2)
        final_mass.append(m_final)
        cont_pbh = 0
        new_label_mass = str("m_")+str(generation)+str(index[cont])
        #print(name_m_1, "\t ", name_m_2,"\t ", "sigma(",name_m_1,",",name_m_2,")")
        #print("{:.2f}".format(m_1.value),"\t","{:.2f}".format(m_2.value),"\t","{:.2e}".format(sigma(m_1,m_2)),"\n")
        #print("N(",name_m_1,")","\t\t N(",name_m_2,")")
        if m_1==m_2:
            N_sp_1 = N_1/2.
            M_sp_1 = N_sp_1 * m_1
            N_sp_2 = N_2/2.
            M_sp_2 = N_sp_2 * m_2
        else:
            N_sp_1 = N_1
            M_sp_1 = N_sp_1 * m_1
            N_sp_2 = N_2
            M_sp_2 = N_sp_2 * m_2
        #print("{:.1f}".format(N_sp_1),"\t\t\t {:.1f}".format(N_sp_2),"\n\n")
        #print("Radius\t| Merger_Rate \t\t| Time to the first Merger | Probability \t| Collision Number")
        #print("--------------------------------------------------------------------------------------------")
        alfa = ((4*24*(2**(1/2))*(R**(2.)))/(math.pi*(G**(5.))*(M_cluster**(5.))*(2**(17./2.))))
        betha = (3./(4*math.pi*(R**(3.))))
        etha = (G*M_cluster/R)
        for h in r :
            r0 = h*u.parsec
            I, err = quad(GAMMA, h+interval, 0., args=(r0,))
            ctes = (14*math.pi/3.)*sigma(m_1,m_2)*(alfa**(-2./7.))*(betha**(9./7.))*(etha**(-12./7.))*(N_sp_1)*(n_3(r0, m_2, M_sp_2)/vel_3(r0, N_sp_2, m_2, M_sp_2, M_cluster))
            merger = (I*ctes*(u.parsec**(3.))).to(u.yr**(-1))
            GAMMA_value = merger
            Prob = GAMMA_value*(t_3_gn*u.yr)#integration dt
            Int_res, err = quad(integrand_n_i, h , h+interval)
            N_sp_r = 3.*(2.**(3./2.))*N_sp_1*(R**(-3.))*Int_res*u.parsec**3.
            Num_Col = N_sp_r*Prob #numero de choques especie i-j
            cont_pbh = cont_pbh + round(float(Num_Col))
            #print("{0:.2f}".format(h),"\t|","{:e}".format(GAMMA_value), "|\t" , "{0:.2f}".format((merger**(-1.)).to(u.Gyr)),"\t\t|\t","{:.4f}".format(Prob), " \t|\t" , "{:.4f}".format(Num_Col), " \t | ")
        #print("\n")
        #print(round(float(cont_pbh)), "PBHs generates of", "{0:.1f}".format(m_final), "denoted by", new_label_mass)
        #print("\n")
        cont+=1
        N_t_final.append(cont_pbh)
        M_t_final.append(cont_pbh*m_final)   
        etiqueta_final.append(new_label_mass)    
    for i, j, k, l in zip(final_mass, M_t_final, etiqueta_final, N_t_final):
        mass_catalog.append(i.value)
        M_tot_catalog.append(j)
        etiqueta_mass.append(k)
        N_tot_catalog.append(l)
    generation +=1
    #print(etiqueta_mass)
    #print(mass_catalog)
    #print(N_tot_catalog)
    final_mass.clear()
    M_t_final.clear()
    N_t_final.clear()
    etiqueta_final.clear()
    #print("\n")

GENERATION NUMBER 3
M_cluster 98630.54408368668 solMass
[('m_i', 'm_i'), ('m_i', 'm_1a'), ('m_i', 'm_2b'), ('m_1a', 'm_1a'), ('m_1a', 'm_2b'), ('m_2b', 'm_2b')] 



In [16]:
N_tot_catalog[0] = N_tot_catalog[0] - (2*N_tot_catalog[3] + N_tot_catalog[4] + N_tot_catalog[5])
N_tot_catalog[1] = N_tot_catalog[1] - (2*N_tot_catalog[6] + N_tot_catalog[7] + N_tot_catalog[4])
N_tot_catalog[2] = N_tot_catalog[2] - (2*N_tot_catalog[8] + N_tot_catalog[5] + N_tot_catalog[7])

In [17]:
N_tot_catalog[1] = N_tot_catalog[1] + N_tot_catalog[3]
N_tot_catalog[2] = N_tot_catalog[2] + N_tot_catalog[4]
N_tot_catalog.pop(4)
N_tot_catalog.pop(3)
M_tot_catalog.pop(4)
M_tot_catalog.pop(3)
etiqueta_mass.pop(4)
etiqueta_mass.pop(3)
mass_catalog.pop(4)
mass_catalog.pop(3)

1.8750000493531436

In [18]:
N_tot_catalog.pop(6)
mass_catalog.pop(6)
etiqueta_mass.pop(6)
M_tot_catalog.pop(6)
N_tot_catalog.pop(4)
mass_catalog.pop(4)
etiqueta_mass.pop(4)
M_tot_catalog.pop(4)

for i in range(len(M_tot_catalog)):
    M_tot_catalog[i] = N_tot_catalog[i]*(mass_catalog[i]*u.M_sun)    
    
for i in N_tot_catalog:
    array_3rd.append(i)
array_0gnd.append(0)
array_1st.append(0)
array_2nd.append(0)

for i in M_tot_catalog:
    array_M_3.append(i)
array_M_0.append(0)
array_M_1.append(0)
array_M_2.append(0)
array_M_3.append(0)

In [ ]:
while generation<5:
    M_cluster = 0
    for i in M_tot_catalog:
        M_cluster += i
    print("GENERATION NUMBER", generation)
    print("M_cluster", M_cluster)
    print(mass_catalog)
    print(list(combinations_with_replacement(etiqueta_mass,2)),"\n")
    cont = 0
    sum_N = 0
    for i,j,k in zip(range(len(list(combinations_with_replacement(mass_catalog,2)))),range(len(list(combinations_with_replacement(etiqueta_mass,2)))), range(len(list(combinations_with_replacement(N_tot_catalog,2))))):
        mass_input = list(combinations_with_replacement(mass_catalog,2))[i]
        name_mass = list(combinations_with_replacement(etiqueta_mass,2))[j]
        N_tot_input = list(combinations_with_replacement(N_tot_catalog,2))[k]
        name_m_1 = name_mass[0]
        m_1=mass_input[0]*u.M_sun
        N_1 = N_tot_input[0]
        name_m_2 = name_mass[1]
        m_2=mass_input[1]*u.M_sun
        N_2 = N_tot_input[1]
        m_final = Total_Mass(m_1, m_2)
        final_mass.append(m_final)
        cont_pbh = 0
        new_label_mass = str("m_")+str(generation)+str(index[cont])
        #print(name_m_1, "\t ", name_m_2,"\t ", "sigma(",name_m_1,",",name_m_2,")")
        #print("{:.2f}".format(m_1.value),"\t","{:.2f}".format(m_2.value),"\t","{:.2e}".format(sigma(m_1,m_2)),"\n")
        #print("N(",name_m_1,")","\t\t N(",name_m_2,")")
        if m_1==m_2:
            N_sp_1 = N_1/2.
            M_sp_1 = N_sp_1 * m_1
            N_sp_2 = N_2/2.
            M_sp_2 = N_sp_2 * m_2
        else:
            N_sp_1 = N_1
            M_sp_1 = N_sp_1 * m_1
            N_sp_2 = N_2
            M_sp_2 = N_sp_2 * m_2
        #print("{:.1f}".format(N_sp_1),"\t\t\t {:.1f}".format(N_sp_2),"\n\n")
        #print("Radius\t| Merger_Rate \t\t| Time to the first Merger | Probability \t| Collision Number")
        #print("--------------------------------------------------------------------------------------------")
        alfa = ((4*24*(2**(1/2))*(R**(2.)))/(math.pi*(G**(5.))*(M_cluster**(5.))*(2**(17./2.))))
        betha = (3./(4*math.pi*(R**(3.))))
        etha = (G*M_cluster/R)
        for h in r :
            r0 = h*u.parsec
            I, err = quad(GAMMA, h+interval, 0., args=(r0,))
            ctes = (14*math.pi/3.)*sigma(m_1,m_2)*(alfa**(-2./7.))*(betha**(9./7.))*(etha**(-12./7.))*(N_sp_1)*(n_3(r0, m_2, M_sp_2)/vel_3(r0, N_sp_2, m_2, M_sp_2, M_cluster))
            merger = (I*ctes*(u.parsec**(3.))).to(u.yr**(-1))
            GAMMA_value = merger
            Prob = GAMMA_value*(t_4_gn*u.yr)#integration dt
            Int_res, err = quad(integrand_n_i, h , h+interval)
            N_sp_r = 3.*(2.**(3./2.))*N_sp_1*(R**(-3.))*Int_res*u.parsec**3.
            Num_Col = N_sp_r*Prob #numero de choques especie i-j
            cont_pbh = cont_pbh + round(float(Num_Col))
            #print("{0:.2f}".format(h),"\t|","{:e}".format(GAMMA_value), "|\t" , "{0:.2f}".format((merger**(-1.)).to(u.Gyr)),"\t\t|\t","{:.4f}".format(Prob), " \t|\t" , "{:.4f}".format(Num_Col), " \t | ")
        #print("\n")
        #print(round(float(cont_pbh)), "PBHs generates of", "{0:.1f}".format(m_final), "denoted by", new_label_mass)
        #print("\n")
        cont+=1
        N_t_final.append(cont_pbh)
        M_t_final.append(cont_pbh*m_final)   
        etiqueta_final.append(new_label_mass)    
    for i, j, k, l in zip(final_mass, M_t_final, etiqueta_final, N_t_final):
        mass_catalog.append(i.value)
        M_tot_catalog.append(j)
        etiqueta_mass.append(k)
        N_tot_catalog.append(l)
    generation +=1
    #print(etiqueta_mass)
    #print(mass_catalog)
    #print(N_tot_catalog)
    final_mass.clear()
    M_t_final.clear()
    N_t_final.clear()
    etiqueta_final.clear()
    #print("\n")

GENERATION NUMBER 4
M_cluster 97369.33587561139 solMass
[1.0, 1.8750000493531436, 2.711956662136698, 3.529306720226462, 4.30981684563379]
[('m_i', 'm_i'), ('m_i', 'm_1a'), ('m_i', 'm_2b'), ('m_i', 'm_3c'), ('m_i', 'm_3e'), ('m_1a', 'm_1a'), ('m_1a', 'm_2b'), ('m_1a', 'm_3c'), ('m_1a', 'm_3e'), ('m_2b', 'm_2b'), ('m_2b', 'm_3c'), ('m_2b', 'm_3e'), ('m_3c', 'm_3c'), ('m_3c', 'm_3e'), ('m_3e', 'm_3e')] 



In [ ]:
N_tot_catalog[0] = N_tot_catalog[0] - (2*N_tot_catalog[5] + N_tot_catalog[6] + N_tot_catalog[7] + N_tot_catalog[8] + N_tot_catalog[9])
N_tot_catalog[1] = N_tot_catalog[1] - (2*N_tot_catalog[10] + N_tot_catalog[11] + N_tot_catalog[12] + N_tot_catalog[13] + N_tot_catalog[6])
N_tot_catalog[2] = N_tot_catalog[2] - (2*N_tot_catalog[14] +  N_tot_catalog[15] + N_tot_catalog[16] + N_tot_catalog[7] + N_tot_catalog[11])
N_tot_catalog[3] = N_tot_catalog[3] - (2*N_tot_catalog[17] +  N_tot_catalog[18] + N_tot_catalog[8] + N_tot_catalog[12] + N_tot_catalog[15])
N_tot_catalog[4] = N_tot_catalog[4] - (2*N_tot_catalog[19] + N_tot_catalog[9] +  N_tot_catalog[13] + N_tot_catalog[16] + N_tot_catalog[18])

In [ ]:
N_tot_catalog[1] = N_tot_catalog[1] + N_tot_catalog[5]
N_tot_catalog[2] = N_tot_catalog[2] + N_tot_catalog[6]
N_tot_catalog[3] = N_tot_catalog[3] + N_tot_catalog[7] + N_tot_catalog[10]
N_tot_catalog[4] = N_tot_catalog[4] + N_tot_catalog[8] + N_tot_catalog[11]
N_tot_catalog[9] = N_tot_catalog[9] + N_tot_catalog[12] + N_tot_catalog[14]
N_tot_catalog[13] = N_tot_catalog[13] + N_tot_catalog[15]
N_tot_catalog[16] = N_tot_catalog[16] + N_tot_catalog[17]
delete_array=[17, 15, 14, 12, 11, 10, 8, 7, 6, 5]

for i in delete_array:
    N_tot_catalog.pop(i)
    mass_catalog.pop(i)
    M_tot_catalog.pop(i)
    etiqueta_mass.pop(i)

In [ ]:
for i in range(len(M_tot_catalog)):
    M_tot_catalog[i] = N_tot_catalog[i]*(mass_catalog[i]*u.M_sun)

for i in M_tot_catalog:
    array_M_4.append(i)
array_M_0.append(0)
array_M_1.append(0)
array_M_2.append(0)
array_M_3.append(0)
    
for i in N_tot_catalog:
    array_4th.append(i)
array_0gnd.append(0)
array_1st.append(0)
array_2nd.append(0)
array_3rd.append(0)

M_cluster = 0
for i in M_tot_catalog:
    M_cluster += i

In [ ]:
print("percentage of mass lost:",(100-(M_cluster/M)*100))

In [ ]:
print(M_cluster/M)

In [ ]:
print(M_tot_catalog[0]/M_cluster)

In [ ]:
age_time = [z_20/u.yr,t_1_gn+z_20/u.yr,t_2_gn + t_1_gn+z_20/u.yr,t_3_gn + t_2_gn + t_1_gn+z_20/u.yr, t_4_gn + t_3_gn + t_2_gn + t_1_gn+z_20/u.yr]

In [ ]:
t_1_gnb = 3.3
t_2_gnb = 2.3
t_3_gnb = 3.3
t_4_gnb = 4.7
age_timeb = [z_20.value,t_1_gnb+z_20.value,t_2_gnb + t_1_gnb+z_20.value,t_3_gnb + t_2_gnb + t_1_gnb+z_20.value, t_4_gnb + t_3_gnb + t_2_gnb + t_1_gnb+z_20.value]

In [ ]:
array_sp_1 = []
array_sp_2 = []
array_sp_3 = []
array_sp_4 = []
array_sp_5 = []

array_sp_1.append(array_0gnd[0])
array_sp_1.append(array_1st[0])
array_sp_1.append(array_2nd[0])
array_sp_1.append(array_3rd[0])
array_sp_1.append(array_4th[0])
array_sp_2.append(array_0gnd[1])
array_sp_2.append(array_1st[1])
array_sp_2.append(array_2nd[1])
array_sp_2.append(array_3rd[1])
array_sp_2.append(array_4th[1])
array_sp_3.append(array_0gnd[2])
array_sp_3.append(array_1st[2])
array_sp_3.append(array_2nd[2])
array_sp_3.append(array_3rd[2])
array_sp_3.append(array_4th[2])
array_sp_4.append(array_0gnd[3])
array_sp_4.append(array_1st[3])
array_sp_4.append(array_2nd[3])
array_sp_4.append(array_3rd[3])
array_sp_4.append(array_4th[3])
array_sp_5.append(array_0gnd[4])
array_sp_5.append(array_1st[4])
array_sp_5.append(array_2nd[4])
array_sp_5.append(array_3rd[4])
array_sp_5.append(array_4th[4])
array_sp_6 = [0,0,0,0,54]
array_sp_7 = [0,0,0,0,13]

In [ ]:
print("Time to:")
print("First Generation:", ("{:.2f}".format((t_1_gn*u.yr).to(u.Gyr))))
print("Second Generation:", "{:.2f}".format(((t_1_gn)*u.yr + t_2_gn*u.yr).to(u.Gyr)))
print("Third Generation:", "{:.2f}".format(((t_1_gn)*u.yr + t_2_gn*u.yr + t_3_gn*u.yr).to(u.Gyr)))
print("Fourth Generation:", "{:.2f}".format(((t_1_gn)*u.yr + t_2_gn*u.yr + t_3_gn*u.yr + t_4_gn*u.yr).to(u.Gyr)))

pl.xlim(0, 13.5)
pl.ylim(0,2e4)
pl.ylabel('Number of PBHs')
pl.xlabel('Gyr')
#pl.plot(age_time, array_sp_1)
pl.plot(age_timeb, array_sp_1, label='_nolegend_')
pl.plot(age_timeb, array_sp_2, color='orange')
pl.plot(age_timeb, array_sp_3, color='violet')
pl.plot(age_timeb, array_sp_4, color='olive')
pl.plot(age_timeb, array_sp_5, color='cyan')
pl.plot(age_timeb, array_sp_6)
pl.plot(age_timeb, array_sp_7)
pl.axvline(x=(t_1_gnb+z_20.value), color="black", linestyle="dashed")
pl.axvline(x=(t_2_gnb+t_1_gnb+z_20.value), color="black", linestyle="dashed")
pl.axvline(x=(t_3_gnb+t_2_gnb+t_1_gnb+z_20.value), color="black", linestyle="dashed")
pl.yscale('symlog')
pl.legend( ('1.8 M$_{\odot}$', '2.7 M$_{\odot}$', '3.5 M$_{\odot}$', '4.3 M$_{\odot}$', '5.1 M$_{\odot}$', '5.8 M$_{\odot}$'), loc = 'upper left')
#pl.savefig('Mass_population_from_1M_sun.png', dpi=300, bbox_inches='tight')
pl.show()